In [71]:
import pandas as pd
import numpy as np
import sklearn
import gc
import os
from scipy import stats

In [110]:
# Default functions:['max', 'min',max_minus_min, 'std', 'mean','median', one_third,two_third]



def numerical_agg( df, gp_col, agg_col, extrafunc_list = None):
    
    agg_list = ['max', 'min', 'std','mean']
    '''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
    if extrafunc_list:
        if 'one_third' in extrafunc_list:
            def one_third(series):
                return series.quantile(1/3)
            
        if 'two_third' in extrafunc_list:
            def two_third(series):
                return series.quantile(2/3)
            
        if 'max_minus_min' in extrafunc_list:
            def max_minus_min(series):
                return (series.max() - series.min())
        
        if 'two_minus_one_third' in extrafunc_list:
#             print('two_minus_one')
#             print('get two minus one')
            def two_minus_one_third(series):
                return(series.quantile(0.66666) - series.quantile(0.33333))
       
        if 'positive_count' in extrafunc_list:
#             print('get positive count')
            def positive_count(series):
                return pd.Series(series > 0).sum()

        if 'negative_count' in extrafunc_list:
#             print('get negative count')
            def negative_count(series):
                return pd.Series(series < 0).sum()

        if 'standard_error' in extrafunc_list:
#             print('get std-error')
            def standard_error(series):
                return series.std()/np.sqrt(len(series))

        if 'normed_std' in extrafunc_list:
#             print('get normed_std')
            def normed_std(series):
                return series.std()/series.mean()

        if 'trimmed_mean_10_pct' in extrafunc_list:
#             print('get_trimmed 10 pct')
            def trimmed_mean_10_pct(series):
                return stats.trim_mean(series.dropna(), 0.1)

        if 'trimmed_mean_25_pct' in extrafunc_list:
#             print('get_trimmed 25 pct')
            def trimmed_mean_25_pct(series):
                return stats.trim_mean(series.dropna(), 0.25)
        
        list_tocall = []
        for i in extrafunc_list:
            if i not in set(['sum', 'median']):
                list_tocall.append(eval(i))
            
    if extrafunc_list != None:   
        agg_list.extend(list_tocall)
    
    _df = df.groupby(gp_col).agg({agg_col:agg_list})
    columns = []
    for pre in _df.columns.levels[0]:
        for middle in _df.columns.levels[1]:
            columns.append('bureau_%s_%s' %(pre,middle))
    _df.columns = columns
    
    return _df.reset_index()
            
        

In [111]:
Path = 'home/kai/qianli/home_credit/qianli/'
test = pd.read_csv('./1.csv')

In [112]:
print(test)

   a    b    c
0  1  NaN  1.0
1  2  2.0  NaN
2  3  3.0  2.0
3  1  2.0  NaN
4  2  2.0  NaN
5  3  5.0  NaN
6  1  4.0  NaN
7  2  5.0  NaN
8  3  4.0  NaN


In [113]:
numerical_agg(test,'a','b', extrafunc_list = ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third'])

get normed_std


,a,bureau_b_max,bureau_b_min,bureau_b_std,bureau_b_mean,bureau_b_two_minus_one_third,bureau_b_positive_count,bureau_b_negative_count,bureau_b_standard_error,bureau_b_trimmed_mean_10_pct,bureau_b_trimmed_mean_25_pct,bureau_b_normed_std,bureau_b_max_minus_min,bureau_b_one_third,bureau_b_two_third
0,1,4.0,2.0,1.414214,3.0,0.66666,2.0,0.0,0.816497,3.0,3.0,0.471405,2.0,2.666667,3.333333
1,2,5.0,2.0,1.732051,3.0,0.99996,3.0,0.0,1.000000,3.0,3.0,0.577350,3.0,2.000000,3.000000
2,3,5.0,3.0,1.000000,4.0,0.66666,3.0,0.0,0.577350,4.0,4.0,0.250000,2.0,3.666667,4.333333
